In [1]:
# from sklearn.neural_network import MLPRegressor
import numpy as np
# Initial imports
import pandas as pd
from pathlib import Path
# from sklearn.datasets import make_blobs, make_moons
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
import hvplot.pandas
import holoviews as hv
from itertools import islice

%matplotlib inline

read the GMA data with the areas and avg's for each
data contains; Actual sales price from the prio year and the pct change, the transactions from the prior year and the pct change, etc.


In [2]:

# Attempt to load the CSV file using a different encoding
data_csv = pd.read_csv(Path("./GMA_data.csv"))

data_csv.head()



,Geo Market Area,ASP 12/2021 - 12/2022,ASP 12/2022 - 12/2023,% Change,Transactions 12/2021 - 12/2022,Transactions 12/2022 - 12/2023,% Change.1,MOI 11/2022,MOI 11/2023,% Change.2,Avg DOM 12/2021 - 12/2022,Avg DOM 12/2022 - 12/2023,% Change.3
0,1960/Cypress,"$305,022","$319,320",4.70%,290,205.0,-29.30%,1.5,2.6,72.10%,15.0,35.7,138.00%
1,1960/Cypress Creek North,"$321,283","$331,624",3.20%,411,245.0,-40.40%,2.2,4.6,107.50%,27.6,39.2,42.00%
2,1960/Cypress Creek South,"$265,879","$258,609",-2.70%,786,613.0,-22.00%,2.2,2.9,28.20%,23.3,40.2,72.50%
3,Aldine Area,"$237,692","$237,042",-0.30%,880,755.0,-14.20%,2.5,2.9,14.60%,23.2,38.1,64.20%
4,Alief,"$260,950","$262,756",0.70%,523,443.0,-15.30%,2.5,3.2,25.50%,21.8,31.6,45.00%


In [3]:
data_csv.dtypes

Geo Market Area                    object
ASP 12/2021 - 12/2022              object
ASP 12/2022 - 12/2023              object
% Change                           object
Transactions 12/2021 - 12/2022      int64
Transactions 12/2022 - 12/2023    float64
% Change.1                         object
MOI 11/2022                        object
MOI 11/2023                        object
% Change.2                         object
Avg DOM 12/2021 - 12/2022         float64
Avg DOM 12/2022 - 12/2023         float64
% Change.3                         object
dtype: object

In [4]:
data_csv.columns

Index(['Geo Market Area', 'ASP 12/2021 - 12/2022', 'ASP 12/2022 - 12/2023',
       '% Change', 'Transactions 12/2021 - 12/2022',
       'Transactions 12/2022 - 12/2023', '% Change.1', 'MOI 11/2022',
       'MOI 11/2023', '% Change.2', 'Avg DOM 12/2021 - 12/2022',
       'Avg DOM 12/2022 - 12/2023', '% Change.3'],
      dtype='object')

In [5]:
#This is the Data That you need for housing data,
data_csv = data_csv.drop(columns=['% Change', '% Change.1', 'MOI 11/2022',
       'MOI 11/2023', '% Change.2' , '% Change.3'])
data_csv.head()

,Geo Market Area,ASP 12/2021 - 12/2022,ASP 12/2022 - 12/2023,Transactions 12/2021 - 12/2022,Transactions 12/2022 - 12/2023,Avg DOM 12/2021 - 12/2022,Avg DOM 12/2022 - 12/2023
0,1960/Cypress,"$305,022","$319,320",290,205.0,15.0,35.7
1,1960/Cypress Creek North,"$321,283","$331,624",411,245.0,27.6,39.2
2,1960/Cypress Creek South,"$265,879","$258,609",786,613.0,23.3,40.2
3,Aldine Area,"$237,692","$237,042",880,755.0,23.2,38.1
4,Alief,"$260,950","$262,756",523,443.0,21.8,31.6


Obtain and read the (general) houston data and retrive head with name of data 
dataset contains ; dollar volume, average price, and median price.

In [6]:
H_data= pd.read_csv(Path("./Houston_data.csv"),skiprows=1)
H_data.dropna(inplace=True)
H_data.head()                   

,Property Type,Listings*,Year Ago,Dollar Volume,Year Ago.1,Price,Year Ago.2,Price.1,Year Ago.3
1,Single Family,"7,983",-4%,"$3,353,852,000",-1%,"$420,124",3%,"$339,000",0%
2,Townhouse/Condominium,556,-19%,"$145,065,478",-17%,"$260,909",3%,"$225,000",3%
3,Multifamily,65,-22%,"$31,514,889",-27%,"$484,844",-6%,"$390,000",-5%
4,Country Homes,244,-10%,"$142,090,041",-8%,"$582,336",2%,"$362,500",-1%
5,High Rise,73,30%,"$35,446,505",21%,"$485,569",-7%,"$342,200",3%


In [7]:
H_data.columns

Index(['Property Type', 'Listings*', 'Year Ago', 'Dollar Volume', 'Year Ago.1',
       'Price', 'Year Ago.2', 'Price.1', 'Year Ago.3'],
      dtype='object')

In [8]:
H_data.dtypes

Property Type    object
Listings*        object
Year Ago         object
Dollar Volume    object
Year Ago.1       object
Price            object
Year Ago.2       object
Price.1          object
Year Ago.3       object
dtype: object

grab the days on market data and make a dataframe for the average DOM

In [9]:
Dom_data = pd.read_csv(Path("./Days_on_market.csv"),skiprows=1)
Dom_data.dropna(inplace=True)
Dom_data.head()

,Property Type,Sqft*,Year Ago,DOM,Year Ago.1,Listings,Year Ago.2,Listings.1,Year Ago.3,Listings.2,Year Ago.4
1,Single Family,$175,0%,51.0,55%,"62,184",-9%,"93,700",-4%,"20,614",41%
2,Townhouse/Condominium,$167,0%,42.0,20%,"4,559",-21%,"7,161",-12%,"1,396",23%
3,Multifamily,$171,-1%,56.0,8%,541,-11%,"1,288",5%,407,44%
4,Country Homes,$397,0%,105.0,8%,"1,932",-14%,"4,280",-1%,"2,654",35%
5,High Rise,$310,4%,60.0,-6%,583,-10%,"1,170",-1%,397,9%


In [10]:
Dom_data.dtypes

Property Type     object
Sqft*             object
Year Ago          object
DOM              float64
Year Ago.1        object
Listings          object
Year Ago.2        object
Listings.1        object
Year Ago.3        object
Listings.2        object
Year Ago.4        object
dtype: object

Grab the Historic data of August: along with the year and year befores data, number of listings, avg/median pricig, and (optional:DOM)
concat the avg and medean pct to the new data frame dataframe, obtain the dtypes

In [11]:
Historic_pricing = pd.read_csv(Path("./Historic_pricing.csv"),skiprows=1)
Historic_pricing.head()

,Year,Month,Listings*,Price,Price.1,Sqft,Listings,Listings.1,Listings.2,DOM,Orig. List Price
0,2021,Jan,"4,815","$321,238","$255,000",$132,"7,202","9,088","6,097",43,96.70%
1,2021,Feb,"4,671","$348,928","$270,000",$139,"6,148","7,781","5,721",42,97.20%
2,2021,Mar,"7,327","$363,397","$285,000",$143,"8,831","7,184","7,889",39,97.80%
3,2021,Apr,"7,459","$372,590","$290,500",$148,"10,164","7,725","8,195",33,98.80%
4,2021,May,"7,856","$389,681","$302,500",$152,"10,280","7,938","8,341",28,99.60%


In [12]:
Historic_pricing.dtypes

Year                 int64
Month               object
Listings*           object
Price               object
Price.1             object
Sqft                object
Listings            object
Listings.1          object
Listings.2          object
DOM                  int64
Orig. List Price    object
dtype: object

(optional) step: grab and scale data and concat the data values to make a new datframe.

Give the User the Option to input a specific area, 
by allowin the user to slice into the concated data; and area data,
Create a If stament for the Geographical area and then populate the model for predictive analysis


In [18]:
H_data.drop(columns=['Year Ago','Year Ago.1','Year Ago.2','Year Ago.3'])
# Prompt the user to enter an area name
user_input_area = input("Enter the name of the area: ")

# Filter the DataFrame for rows where the 'Geo Market Area' matches the user input
data_csv = data_csv[data_csv['Geo Market Area'] == user_input_area]

#allow  a sinput on the prev data  to pull the property type, and sqftg
user_input_area_sqft = input("user_input_area")
Dom_data = Dom_data[Dom_data['Property Type', 'Sqft*']==user_input_area]
# Display the filtered data
print(data_csv)

#create a if statement that call for the avg price for data in a new dataframe

for index, row in islice(data_csv.iterrows(), 1, None):

   if index == (number_of_rows):
        break 


KeyError: ('Property Type', 'Sqft*')

In [15]:
# H_data = H_data.rename(columns={0: 'Property ', 1: 'Price', 2: 'Dollar '})

# H_data.reset_index(level=None, drop=False, inplace=False, col_level=3, col_fill=3)
# print(df)
# columns_to_drop = ["Unnamed" , "Sqft*"]
# Dom_data = columns_to_drop("Unnamed")